In [1]:
# Libraries
import numpy as np
import pandas as pd
import re

## Import crime stats from top 15 populated cities

In [2]:
##Declare a function that receives an xls file and extracts for it the data needed for our dataframe
def clean_xls(trimester, xls_file):
    
    excel_crimes = pd.read_excel(xls_file)
 
    if xls_file[:4] == '2019' or xls_file[:4] == '2018':
        crimes = excel_crimes.iloc[6:231,].reset_index(drop=True)
    
    else:
        crimes = excel_crimes.iloc[6:216,].reset_index(drop=True)
    crimes.columns = ['crime', 'quantity']
    crimes['year'] = re.search(r'\d{4}', xls_file)[0]
    crimes['trimester'] = trimester
    crimes['city'] = crimes['crime'].apply(lambda x: re.findall(r'- Municipio de (.*)', x))
    crimes['city'] = crimes['city'].apply(lambda x: x[0] if x else np.nan)
    crimes['city'] = crimes['city'].ffill()
    crimes = crimes.dropna()
    crimes['quantity'] = crimes['quantity'].astype('int')
    
    return crimes


##Declare a function that receives a dataframe and trims it by year and trimester to operate with the new DF
def trim(df, year):
    
    if year == '2019':
        n = 4
    
    else:
        n = 5
    
    lst = []
    for i in range(1,n):
        trim = df[df['trimester'] == i]
        lst.append(trim)
    
    return lst


#Declare a function that separates the year data by trimester and dissagregates the total of crimes by substracting the 
#number of crimes of the previous period from the current number of crimes
def fix_number_crimes(df):
    
    for i in df.year.unique():
        temp_df = df[df.year == i]
    
        if i == '2019':
           
            trim1, trim2, trim3 = trim(temp_df, i)
            temp3 = trim3['quantity'].reset_index(drop=True) - trim2['quantity'].reset_index(drop=True)
            temp3.index = trim3.index
            trim3['quantity'] = temp3
            
            temp2 = trim2['quantity'].reset_index(drop=True) - trim1['quantity'].reset_index(drop=True)
            temp2.index = trim2.index
            trim2['quantity'] = temp2
            
            df.update(trim3)
            df.update(trim2)
        
        else:
            trim1, trim2, trim3, trim4 = trim(temp_df, i)  
            temp4 = trim4['quantity'].reset_index(drop=True) - trim3['quantity'].reset_index(drop=True)
            temp4.index = trim4.index
            trim4['quantity'] = temp4
            
            temp3 = trim3['quantity'].reset_index(drop=True) - trim2['quantity'].reset_index(drop=True)
            temp3.index = trim3.index
            trim3['quantity'] = temp3
            
            temp2 = trim2['quantity'].reset_index(drop=True) - trim1['quantity'].reset_index(drop=True)
            temp2.index = trim2.index
            trim2['quantity'] = temp2
            
            df.update(trim4)
            df.update(trim3)
            df.update(trim2)
        

#Declare a list with the files to process, a list with the semesters they belong to, and a dataframe to store the info      
list_xls = ['2019_1T.xls', '2019_2T.xls', '2019_3T.xls', '2018_1T.xls', '2018_2T.xls', '2018_3T.xls', '2018_4T.xls', '2017_1T.xls', '2017_2T.xls', '2017_3T.xls', '2017_4T.xls', '2016_1T.xls', '2016_2T.xls', '2016_3T.xls', '2016_4T.xls']
list_trimester = [1,2, 3] + [1,2,3,4]*3
crimes_spain = pd.DataFrame()

#Clean each xls
for i, j in zip(list_trimester, list_xls):
    crimes_spain = pd.concat([crimes_spain, clean_xls(i, j)])

#Reset the index to ensure the update method can be properly used in the fix_number_crimes() function
crimes_spain = crimes_spain.reset_index(drop=True)
fix_number_crimes(crimes_spain)

C:\Users\miner\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\miner\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\miner\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [3]:
crimes_spain.head()

,crime,quantity,year,trimester,city
0,1.-Homicidios dolosos y asesinatos consumados,0.0,2019,1.0,Córdoba
1,2.-Homicidios dolosos y asesinatos en grad...,0.0,2019,1.0,Córdoba
2,3.-Delitos graves y menos graves de lesion...,18.0,2019,1.0,Córdoba
3,4.-Secuestro,1.0,2019,1.0,Córdoba
4,5.-Delitos contra la libertad e indemnidad...,32.0,2019,1.0,Córdoba


In [4]:
crimes_spain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3030 entries, 0 to 3029
Data columns (total 5 columns):
crime        3030 non-null object
quantity     3030 non-null float64
year         3030 non-null object
trimester    3030 non-null float64
city         3030 non-null object
dtypes: float64(2), object(3)
memory usage: 118.5+ KB


In [5]:
##Change types
crimes_spain['trimester'] = crimes_spain['trimester'].astype(int)
crimes_spain['quantity'] = crimes_spain['quantity'].astype(int)
crimes_spain['year'] = crimes_spain['year'].astype(int)

In [6]:
crimes_spain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3030 entries, 0 to 3029
Data columns (total 5 columns):
crime        3030 non-null object
quantity     3030 non-null int32
year         3030 non-null int32
trimester    3030 non-null int32
city         3030 non-null object
dtypes: int32(3), object(2)
memory usage: 83.0+ KB


In [7]:
##Normalize dataset names to meet  the ones in the population table
crimes_spain['city'].replace('Palma de Mallorca', 'Palma', inplace = True)
crimes_spain['city'].replace('Alicante/Alacant', 'Alicante', inplace = True)
crimes_spain['city'].replace('Palmas de Gran Canaria (Las)', 'Las Palmas de Gran Canaria', inplace = True)

In [8]:
crimes_spain.head()

,crime,quantity,year,trimester,city
0,1.-Homicidios dolosos y asesinatos consumados,0,2019,1,Córdoba
1,2.-Homicidios dolosos y asesinatos en grad...,0,2019,1,Córdoba
2,3.-Delitos graves y menos graves de lesion...,18,2019,1,Córdoba
3,4.-Secuestro,1,2019,1,Córdoba
4,5.-Delitos contra la libertad e indemnidad...,32,2019,1,Córdoba


## Create CSV for analysis

In [9]:
crimes_spain.to_csv('../../Analysis/crimes_spain.csv')